# Convergence ML Engineer / Researcher Take-home

We'd like to learn a little more about how you practically approach a small research-like project loosely based on Rejection Sampling Fine-tuning (aka RFT, introduced in https://arxiv.org/abs/2308.01825).

Tip: focus on section 3.3 ("Rejection Sampling Fine-tuning"). The paper isn't the best written, and we're happy to clarify anything.

We will provide some skeleton code for you to guide what we would like to see from you, although if you have ideas for a different structure you feel is better or more elegant, then feel free to rewrite and replace at will.

Note: your final submission does not have to be in a colab notebook, does not have to use Hugging Face, etc.

---

## A note from the team

We want to give you a chance to show off some of your best abilities.

For some people that might mean generating high quality data in a smart way. For others, it might be speeding up the whole process to enable easy reproducibility, and maybe organizing the code in a better way than given. Yet for others, it might be a chance to show off some modern policy optimization techniques like DPO or its variants. Or maybe focusing on solid evaluations and identifying limitations of small models and limited fine-tuning.

An ideal outcome of course is some sense of the model improving its mathematical abilities, but it’s not a bad thing if the final evaluation somehow shows equal or worse performance 😂 (negative results are results).

Ask lots of question! We're happy to answer any questions about the assignment, and to discuss concepts like RFT.

# Setup [ignore - just run]
***

In [6]:
#!pip3 install -r requirements.txt

In [ ]:
import os  
import torch
import random 
import datasets
import numpy as np 
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from data import GSM8KDataset
import utils

os.environ["TOKENIZERS_PARALLELISM"] = "true"
SEED = 128 
MODEL_NAME = "microsoft/Phi-3.5-mini-instruct"

# set seeds
torch.random.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

/Users/ericliu/research/rft-gsm8k/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16, # accelerate inf 
    trust_remote_code=True,
    attn_implementation="flash_attention_2",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_dataset = datasets.load_dataset('gsm8k', 'main')['train']
val_dataset = datasets.load_dataset('gsm8k', 'main')['test'] 
print(f"Num Training instances: {len(train_dataset)}")
print(f"Num Validation instances: {len(val_dataset)}")


A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install 'accelerate>=0.26.0'`

# Dataset Inspection 
****

In [ ]:
train_dataset

In [ ]:
for _ in range(5):
    seed = np.random.randint(0, len(train_dataset))
    print("*"*100)
    print(f"Checking instance {seed}:")
    utils.inspect_instance(train_dataset, seed)

## Extract statistics 

We only look at train set now for certain information that will be used
during inference 

- Maximum length (num_tokens) of question: 239
- Maximum length (num_tokens) of answer: 475 

In [ ]:
train_dataset = train_dataset.map(
    lambda x: utils.GSM8KParser.get_question_length(x['question'], tokenizer)
)

train_dataset = train_dataset.map(
    lambda x: utils.GSM8KParser.get_answer_length(x['answer'], tokenizer) 
)
print(f"Maximum answer num_tokens: {max(train_dataset['answer_length'])}")
print(f"Maximum question num_tokens: {max(train_dataset['question_length'])}")

## Extract Answer 

In [ ]:
# infer number of hops 
train_dataset = train_dataset.map(
    lambda x: utils.GSM8KParser.get_num_hops(x['answer'])
)

# infer answes using ground truth parser 
train_dataset = train_dataset.map(
    lambda x: utils.GSM8KParser.get_answer_from_gt(x['answer'])
)

In [ ]:
# Optinal Cell (Only to verify that parsing from 
# ground truth and parsing from completion would 
# yield the same result 
# infer answers using prediction parser
answer_str_inf = [
    utils.GSM8KParser.get_answer_from_pred(x)['answer_str_digit'] \
    for x in train_dataset['answer']
]
assert answer_str_inf == train_dataset['answer_str_digit']

# Model Vibe Check 

# Generating Synthetic Data
***

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

ds = datasets.load_dataset("openai/gsm8k", "main")

def generate_synthetic_data(
    model: AutoModelForCausalLM,
    tokenizer: AutoTokenizer,
    dataset: datasets.Dataset,
) -> datasets.Dataset:
    questions = dataset['question']

    # Replace the next couple of lines with your own code to generate both a
    # correct solution and an incorrect example from the model. You should
    # generate 10 examples for each question and pick (one of) the best
    # and worst respectively.

    solutions = ['' for q in questions]
    incorrect_examples = ['' for q in questions]

    return datasets.Dataset.from_dict(
      {
          "question": questions,
          "solution": solutions,
          "incorrect_example": incorrect_examples,
      }
    )

synthetic_dataset = generate_synthetic_data(model, tokenizer, ds['train'])

### Training on the Collected Data

Of course, the computational resources of colab are limited.

Employ whatever trick you would like to reduce the VRAM requirements during training (including swapping the model for a smaller one, although please only as a last resort).

In [ ]:
from dataclasses import dataclass
from torch.optim import AdamW
from torch.utils.data import DataLoader

@dataclass
class TrainConfig:
    lr: float = 3e-5
    epochs: int = 2
    batch_size: int = 4
    device: str = 'cpu'

def train(
    model: AutoModelForCausalLM,
    tokenizer: AutoTokenizer,
    dataset: datasets.Dataset,
    config: TrainConfig,
) -> AutoModelForCausalLM:

    def collate_fn(batch):
        # Implement this
        return

    def loss_fn(batch):
        # Implement an appropriate loss - note we don't expect this to necessarily
        # be tied to the earlier mentioned paper, just something that is sensible
        return

    optimizer = AdamW(model.parameters(), lr=config.lr)

    dataloader = DataLoader(dataset, collate_fn=collate_fn, batch_size=config.batch_size, shuffle=True)

    # This is a pretty bare-bones loop, feel free to add anything particularly useful
    for epoch in config.epochs:
        model.train()

        for batch in dataloader:
            optimizer.zero_grad()

            loss = loss_fn(**batch)
            loss.backward()

            optimizer.step()

    return model

amazing_model = train(model, tokenizer, synthetic_dataset, TrainConfig())

### Evaluating the Model

This final part is more free-form. We'd like to evaluate our new model on the test set to see if it's improved, but then spend however much time you have left examining the model more closely / demonstrating some interesting behaviour / showing off beautiful plots.


In [ ]:
def evaluate_model(
    model: AutoModelForCausalLM,
    eval_dataset: datasets.Dataset,
) -> float:
    return 0.0

our_score = evaluate_model(amazing_model, ds['test'])
original_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
their_score = evaluate_model(original_model, ds['test'])

conclusion = '🎉🎉🎉' if our_score > their_score else 'oh well, was it even supposed to work?'
print(conclusion)

### [Optional] - Discussion

We would be interested to know:

1.   If you were less time / computationally constrained, what would you do differently?
2.   What would your ideal first project look like if you joined?

